In [ ]:
from libstdm.common.datasets import findspark
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.storagelevel import *
import pandas as pd
from pyspark.sql import Window
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
##回滚测试

In [13]:
spark = findspark.get_spark_session(
    "analyze-safety-broadcast-scv",
    is_local=False
)

2025-11-14 11:42:50,110 [INFO] [findspark.py:62] set spark.yarn.dist.archives = hdfs://DClusterNmg3/user/xinsi_traffic/libstdm/libstdm-hello-latest.tgz#conda
2025-11-14 11:42:50,111 [INFO] [findspark.py:74] set spark yarn.queue = root.xinyewusibu_lukuang_dev
2025-11-14 11:42:50,112 [INFO] [findspark.py:78] set spark.jars = hdfs://DClusterNmg3/user/xinsi_traffic/libstdm/jars/pytraffic.jar


Application Id: application_1758876150248_24060908, Tracking URL: http://bigdata-nmg-hdprm00.nmg01:8088/proxy/application_1758876150248_24060908/


In [18]:
spark

### 2025-11-11 一次下发量、播放量、播放透出率

#### 每个城市下发量、播报量

In [4]:
df =spark.sql("""
SELECT
            dt,
            city_id,
            city_name,
            -- 首次一次播报
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 0 THEN broadcast_count
                    ELSE 0
                END
            ) AS first_brd_cnt,
            -- 一次下发
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 0 THEN server_count
                    ELSE 0
                END
            ) AS first_server_cnt
        FROM
            traffic_online.dwd_map_traffic_safty_bdc_count_di
        WHERE
            dt = '2025-11-11'
        GROUP BY
            dt,
            city_id,
            city_name
        having
            (
                 first_brd_cnt + first_server_cnt 
            ) != 0
            order by city_id
            limit 100
""").toPandas()
print('每个城市下发量、播报量')
print(df)

每个城市下发量、播报量
            dt  city_id city_name  first_brd_cnt  first_server_cnt
0   2025-11-11        0      None           9236             34434
1   2025-11-11        1       北京市        3407044          23900631
2   2025-11-11        2       深圳市        1680337          12350920
3   2025-11-11        3       广州市        3024505          17476471
4   2025-11-11        4       上海市        2560031          16742195
5   2025-11-11        5       杭州市        1152144           7896048
6   2025-11-11        6       武汉市        1170476           5902638
7   2025-11-11        7       天津市        1336831           6077948
8   2025-11-11        8       沈阳市         296607           1428205
9   2025-11-11        9       郑州市         646260           3659198
10  2025-11-11       10       西安市        1181114           4992569
11  2025-11-11       11       南京市         766196           5081325
12  2025-11-11       12       济南市         384082           2270576
13  2025-11-11       13       青岛市         846036  

#### 所有城市播报量、下发量

In [5]:
df =spark.sql("""
SELECT
            dt,
            -- 首次一次播报
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 0 THEN broadcast_count
                    ELSE 0
                END
            ) AS first_brd_cnt,
            -- 一次下发
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 0 THEN server_count
                    ELSE 0
                END
            ) AS first_server_cnt
        FROM
            traffic_online.dwd_map_traffic_safty_bdc_count_di
        WHERE
            dt = '2025-11-11'
        group by dt
        having
            (
                 first_brd_cnt + first_server_cnt 
            ) != 0
            limit 100
""").toPandas()
print('所有城市播报量、下发量')
print(df)

所有城市播报量、下发量
           dt  first_brd_cnt  first_server_cnt
0  2025-11-11       53475494         279623563


#### 每个城市每公里播报量、下发量、播放透出率

In [6]:
df = spark.sql("""
SELECT
    a.dt,
    a.city_name,
    -- 每公里下发量
    sum(a.first_server_cnt) / sum(b.sec_finish_order_dis) AS km_server_cnt,
    -- 每公里播报量
    sum(a.first_brd_cnt) / sum(b.sec_finish_order_dis) AS km_brd_cnt,
    -- 播放透出率 = 总播报次数 / 总下发次数
    sum(a.first_brd_cnt) / sum(a.first_server_cnt) AS server_brd_rate
FROM
(SELECT
            dt,
            city_id,
            city_name,
            -- 首次一次播报
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 0 THEN broadcast_count
                    ELSE 0
                END
            ) AS first_brd_cnt,
            -- 一次下发
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 0 THEN server_count
                    ELSE 0
                END
            ) AS first_server_cnt
        FROM
            traffic_online.dwd_map_traffic_safty_bdc_count_di
        WHERE
            dt = '2025-11-11'
        GROUP BY
            dt,
            city_id,
            city_name
        having
            (
                 first_brd_cnt + first_server_cnt 
            ) != 0
            order by city_id) a

JOIN
(SELECT
            dt,
            city_id,
            SUM(sec_finish_order_dis) AS sec_finish_order_dis,
            COUNT(DISTINCT order_id) AS order_count
        FROM
            (
                SELECT
                    dt,
                    city_id,
                    biz_type,
                    order_id,
                    sec_finish_order_dis
                FROM
                    traffic_online.dwd_map_traffic_safty_bdc_count_di
                WHERE
                    dt >= "2025-11-11"
                GROUP BY
                    dt,
                    city_id,
                    biz_type,
                    order_id,
                    sec_finish_order_dis
                    ) aa
                    GROUP BY
                    dt,
                    city_id) b

ON 
        a.dt = b.dt
        AND a.city_id = b.city_id
        group by a.dt ,a.city_name
        limit 100
""").toPandas()
print("每个城市每公里播报量、下发量、播放透出率")
print(df)

每个城市每公里播报量、下发量、播放透出率
            dt    city_name  km_server_cnt  km_brd_cnt  server_brd_rate
0   2025-11-11          银川市       0.919990    0.221858         0.241153
1   2025-11-11          忻州市       0.738699    0.222867         0.301702
2   2025-11-11          北京市       1.150773    0.164043         0.142550
3   2025-11-11    文山壮族苗族自治州       0.959619    0.234564         0.244435
4   2025-11-11        齐齐哈尔市       0.784619    0.216639         0.276108
5   2025-11-11          松原市       0.774786    0.204420         0.263840
6   2025-11-11          铜川市       1.023962    0.253893         0.247951
7   2025-11-11          莱芜市       1.299568    0.090351         0.069524
8   2025-11-11          安康市       0.874046    0.247803         0.283513
9   2025-11-11          内江市       0.920732    0.191164         0.207622
10  2025-11-11    西双版纳傣族自治州       1.012255    0.260114         0.256965
11  2025-11-11         葫芦岛市       0.770514    0.201221         0.261152
12  2025-11-11          清远市       1.109601 

#### 所有城市每公里播报量、下发量、播放透出率

In [15]:
df = spark.sql("""
SELECT
    a.dt,
    -- 每公里下发量
    sum(a.first_server_cnt) / sum(b.sec_finish_order_dis) AS km_server_cnt,
    -- 每公里播报量
    sum(a.first_brd_cnt) / sum(b.sec_finish_order_dis) AS km_brd_cnt,
    -- 播放透出率 = 总播报次数 / 总下发次数
    sum(a.first_brd_cnt) / sum(a.first_server_cnt) AS server_brd_rate
FROM
(SELECT
            dt,
            city_id,
            city_name,
            -- 首次一次播报
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 0 THEN broadcast_count
                    ELSE 0
                END
            ) AS first_brd_cnt,
            -- 一次下发
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 0 THEN server_count
                    ELSE 0
                END
            ) AS first_server_cnt
        FROM
            traffic_online.dwd_map_traffic_safty_bdc_count_di
        WHERE
            dt = '2025-11-11'
        GROUP BY
            dt,
            city_id,
            city_name
        having
            (
                 first_brd_cnt + first_server_cnt 
            ) != 0
            order by city_id) a

JOIN
(SELECT
            dt,
            city_id,
            SUM(sec_finish_order_dis) AS sec_finish_order_dis,
            COUNT(DISTINCT order_id) AS order_count
        FROM
            (
                SELECT
                    dt,
                    city_id,
                    biz_type,
                    order_id,
                    sec_finish_order_dis
                FROM
                    traffic_online.dwd_map_traffic_safty_bdc_count_di
                WHERE
                    dt = "2025-11-11"
                GROUP BY
                    dt,
                    city_id,
                    biz_type,
                    order_id,
                    sec_finish_order_dis
                    ) aa
                    GROUP BY
                    dt,
                    city_id) b

ON 
        a.dt = b.dt
        AND a.city_id = b.city_id
        group by a.dt
        limit 100
""").toPandas()
print("所有城市每公里播报量、下发量、播放透出率")
print(df)

所有城市每公里播报量、下发量、播放透出率
           dt  km_server_cnt  km_brd_cnt  server_brd_rate
0  2025-11-11       1.060689    0.202847         0.191241


#### 原始代码

In [17]:
df = spark.sql("""
SELECT
    a.*,
    CASE
        WHEN a.biz_type IN (258, 260, 1300, 313, 314, 307, 8000) THEN '网约车'
        WHEN a.biz_type = 259 THEN '顺风车'
        WHEN a.biz_type = 430
        OR (
            a.biz_type BETWEEN 60000
            AND 65000
        ) THEN '花小猪和三方'
        ELSE '其他'
    END AS biz_type_group,
    b.sec_finish_order_dis,
    b.order_count
FROM
    (
        SELECT
            dt,
            city_id,
            city_name,
            biz_type,
            strategy_id,
            -- 计算总的播报次数
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = -1
                    AND is_sec_broad = -1 THEN broadcast_count
                    ELSE 0
                END
            ) AS tot_brd_cnt,
            -- 一次播报
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 0 THEN broadcast_count
                    ELSE 0
                END
            ) AS first_brd_cnt,
            -- 二次播报
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 1 THEN broadcast_count
                    ELSE 0
                END
            ) AS secend_brd_cnt,
            -- 视觉播报
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 1
                    AND is_sec_broad = 0 THEN broadcast_count
                    ELSE 0
                END
            ) AS sight_brd_cnt,
            -- 总下发
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = -1
                    AND is_sec_broad = -1 THEN server_count
                    ELSE 0
                END
            ) AS tot_server_cnt,
            -- 一次下发
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 0 THEN server_count
                    ELSE 0
                END
            ) AS first_server_cnt,
            -- 二次下发
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 1 THEN server_count
                    ELSE 0
                END
            ) AS secend_server_cnt,
            -- 视觉下发
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 1
                    AND is_sec_broad = 0 THEN server_count
                    ELSE 0
                END
            ) AS sight_server_cnt
        FROM
            traffic_online.dwd_map_traffic_safty_bdc_count_di
        WHERE
            dt = "2025-11-11"
        GROUP BY
            dt,
            city_id,
            city_name,
            biz_type,
            strategy_id
        having
            (
                tot_brd_cnt + first_brd_cnt + secend_brd_cnt + sight_brd_cnt + tot_server_cnt + first_server_cnt + secend_server_cnt + sight_server_cnt
            ) != 0
    ) a
    JOIN (
        SELECT
            dt,
            city_id,
            biz_type,
            SUM(sec_finish_order_dis) AS sec_finish_order_dis,
            COUNT(DISTINCT order_id) AS order_count
        FROM
            (
                SELECT
                    dt,
                    city_id,
                    biz_type,
                    order_id,
                    sec_finish_order_dis
                FROM
                    traffic_online.dwd_map_traffic_safty_bdc_count_di
                WHERE
                    dt >= "2025-11-11"
                GROUP BY
                    dt,
                    city_id,
                    biz_type,
                    order_id,
                    sec_finish_order_dis
            ) aa
        GROUP BY
            dt,
            city_id,
            biz_type
    ) b ON (
        a.dt = b.dt
        AND a.city_id = b.city_id
        AND a.biz_type = b.biz_type
    )
UNION ALL
SELECT
    a.*,
    CASE
        WHEN a.biz_type IN (258, 260, 1300, 313, 314, 307, 8000) THEN '网约车'
        WHEN a.biz_type = 259 THEN '顺风车'
        WHEN a.biz_type = 430
        OR (
            a.biz_type BETWEEN 60000
            AND 65000
        ) THEN '花小猪和三方'
        ELSE '其他'
    END AS biz_type_group,
    b.sec_finish_order_dis,
    b.order_count
FROM
    (
        SELECT
            dt,
            city_id,
            city_name,
            biz_type,
            CASE
                WHEN strategy_id IN (
                    '8011',
                    '90020',
                    '90030',
                    '90050',
                    '90040',
                    '20001',
                    '70001'
                ) THEN '路段'
                ELSE '路口'
            END AS strategy_id,
            -- 计算总的播报次数
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = -1
                    AND is_sec_broad = -1 THEN broadcast_count
                    ELSE 0
                END
            ) AS tot_brd_cnt,
            -- 一次播报
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 0 THEN broadcast_count
                    ELSE 0
                END
            ) AS first_brd_cnt,
            -- 二次播报
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 1 THEN broadcast_count
                    ELSE 0
                END
            ) AS secend_brd_cnt,
            -- 视觉播报
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 1
                    AND is_sec_broad = 0 THEN broadcast_count
                    ELSE 0
                END
            ) AS sight_brd_cnt,
            -- 总下发
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = -1
                    AND is_sec_broad = -1 THEN server_count
                    ELSE 0
                END
            ) AS tot_server_cnt,
            -- 一次下发
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 0 THEN server_count
                    ELSE 0
                END
            ) AS first_server_cnt,
            -- 二次下发
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 0
                    AND is_sec_broad = 1 THEN server_count
                    ELSE 0
                END
            ) AS secend_server_cnt,
            -- 视觉下发
            SUM(
                CASE
                    WHEN model_version = 3
                    AND is_sight = 1
                    AND is_sec_broad = 0 THEN server_count
                    ELSE 0
                END
            ) AS sight_server_cnt
        FROM
            traffic_online.dwd_map_traffic_safty_bdc_count_di
        WHERE
            dt = "2025-11-11"
        GROUP BY
            dt,
            city_id,
            city_name,
            biz_type,
            strategy_id
    ) a
    JOIN (
        SELECT
            dt,
            city_id,
            biz_type,
            SUM(sec_finish_order_dis) AS sec_finish_order_dis,
            COUNT(DISTINCT order_id) AS order_count
        FROM
            (
                SELECT
                    dt,
                    city_id,
                    biz_type,
                    order_id,
                    sec_finish_order_dis
                FROM
                    traffic_online.dwd_map_traffic_safty_bdc_count_di
                WHERE
                    dt = "2025-11-11"
                GROUP BY
                    dt,
                    city_id,
                    biz_type,
                    order_id,
                    sec_finish_order_dis
            ) aa
        GROUP BY
            dt,
            city_id,
            biz_type
    ) b ON (
        a.dt = b.dt
        AND a.city_id = b.city_id
        AND a.biz_type = b.biz_type
    )
limit 100
""").toPandas()
print(df)

            dt  city_id city_name  biz_type strategy_id  tot_brd_cnt  \
0   2025-11-11        1       北京市      1300       80011        12272   
1   2025-11-11        1       北京市      1300        1060         7293   
2   2025-11-11        1       北京市      1300       20001       137269   
3   2025-11-11        1       北京市      1300       80010          412   
4   2025-11-11        1       北京市      1300       80005        22417   
5   2025-11-11        1       北京市      1300       80009         2057   
6   2025-11-11        1       北京市      1300        1061        33200   
7   2025-11-11        1       北京市      1300       90030        68436   
8   2025-11-11        1       北京市      1300        8011        35889   
9   2025-11-11        1       北京市      1300       80004         2563   
10  2025-11-11        1       北京市      1300       80015         1218   
11  2025-11-11        1       北京市      1300       80012       155194   
12  2025-11-11        1       北京市      1300       80006        4

### 2025-11-11 干预有效率

#### 原始代码

In [29]:
df = spark.sql("""
SELECT
    dt,
    city_id,
    city_id as city_name,
    -- 无播报
    SUM(
        IF(
            t.is_first_broad IS NULL
            AND t.is_sec_broad IS NULL,
            1,
            0
        )
    ) AS cnt_no_broadcast,
    -- 总播报（两个都为1）
    SUM(
        IF(
            t.is_first_broad = 1
            AND t.is_sec_broad = 1,
            1,
            0
        )
    ) AS cnt_total_broadcast,
    -- 一次播报（只有一次）
    SUM(
        IF(
            t.is_first_broad = 1
            AND (
                t.is_sec_broad IS NULL
                OR t.is_sec_broad = 0
            ),
            1,
            0
        )
    ) AS cnt_first_broadcast,
    SUM(
        IF(
            t.is_sec_broad = 1
            AND (
                t.is_first_broad IS NULL
                OR t.is_first_broad = 0
            ),
            1,
            0
        )
    ) AS cnt_second_broadcast,
    -- 带 speed_trend 的情况
    SUM(
        IF(
            t.is_first_broad IS NULL
            AND t.is_sec_broad IS NULL
            AND t.speed_trend < 0,
            1,
            0
        )
    ) AS cnt_no_broadcast_with_slowdown,
    SUM(
        IF(
            t.is_first_broad = 1
            AND t.is_sec_broad = 1
            AND t.speed_trend < 0,
            1,
            0
        )
    ) AS cnt_total_broadcast_with_slowdown,
    SUM(
        IF(
            t.is_first_broad = 1
            AND (
                t.is_sec_broad IS NULL
                OR t.is_sec_broad = 0
            )
            AND t.speed_trend < 0,
            1,
            0
        )
    ) AS cnt_first_broadcast_with_slowdown,
    SUM(
        IF(
            t.is_sec_broad = 1
            AND (
                t.is_first_broad IS NULL
                OR t.is_first_broad = 0
            )
            AND t.speed_trend < 0,
            1,
            0
        )
    ) AS cnt_second_broadcast_with_slowdown
FROM
    (
        SELECT
            dt,
            city_id,
            order_id,
            driver_id,
            risk_id,
            min(speed_trend) as speed_trend,
            max(is_first_broad) as is_first_broad,
            max(is_sec_broad) as is_sec_broad
        FROM
            traffic_online.dwd_map_traffic_safety_driver_behavior_detail_di
        WHERE
            dt = "2025-11-11"
        group by
            dt,
            city_id,
            order_id,
            driver_id,
            risk_id
    ) t
GROUP BY
    dt,
    city_id
limit 100
""").toPandas()
print(df)

            dt  city_id  city_name  cnt_no_broadcast  cnt_total_broadcast  \
0   2025-11-11      364        364               671                  107   
1   2025-11-11      260        260             13184                 1318   
2   2025-11-11       26         26            210164                 6652   
3   2025-11-11      365        365              1917                  161   
4   2025-11-11      124        124             41743                 3369   
5   2025-11-11      110        110             39768                 1983   
6   2025-11-11       35         35            394826                16356   
7   2025-11-11       21         21            676370                39662   
8   2025-11-11      143        143             79804                 4785   
9   2025-11-11       46         46            145599                10585   
10  2025-11-11      306        306             46726                 2984   
11  2025-11-11      129        129             40552                 1946   

#### 每个城市干预有效率

In [31]:
df = spark.sql("""
SELECT
    dt,
    city_id,
    city_name,
    cnt_no_broadcast,
    cnt_total_broadcast,
    cnt_first_broadcast,
    cnt_second_broadcast,
    cnt_no_broadcast_with_slowdown,
    cnt_total_broadcast_with_slowdown,
    cnt_first_broadcast_with_slowdown,
    cnt_second_broadcast_with_slowdown,
    -- 无播报减速率
    ROUND(IF(cnt_no_broadcast = 0, 0, cnt_no_broadcast_with_slowdown / cnt_no_broadcast * 100), 2) AS no_broadcast_slow_rate,
    -- 一次播报减速率
    ROUND(IF(cnt_first_broadcast = 0, 0, cnt_first_broadcast_with_slowdown / cnt_first_broadcast * 100), 2) AS first_broadcast_slow_rate,
    -- 二次播报减速率
    ROUND(IF(cnt_second_broadcast = 0, 0, cnt_second_broadcast_with_slowdown / cnt_second_broadcast * 100), 2) AS second_broadcast_slow_rate,
    -- 总播报减速率
    ROUND(IF(cnt_total_broadcast = 0, 0, cnt_total_broadcast_with_slowdown / cnt_total_broadcast * 100), 2) AS total_broadcast_slow_rate
FROM (
    SELECT
        dt,
        city_id,
        city_id as city_name,
        -- 无播报风险点数
        SUM(IF(t.is_first_broad IS NULL AND t.is_sec_broad IS NULL, 1, 0)) AS cnt_no_broadcast,
        --总播报点数
        SUM(IF(t.is_first_broad = 1 AND t.is_sec_broad = 1, 1, 0)) AS cnt_total_broadcast,
        -- 仅一次播报数
        SUM(IF(t.is_first_broad = 1 AND (t.is_sec_broad IS NULL OR t.is_sec_broad = 0), 1, 0)) AS cnt_first_broadcast,
        -- 仅二次播报数
        SUM(IF(t.is_sec_broad = 1 AND (t.is_first_broad IS NULL OR t.is_first_broad = 0), 1, 0)) AS cnt_second_broadcast,
        -- 无播报减速数
        SUM(IF(t.is_first_broad IS NULL AND t.is_sec_broad IS NULL AND t.speed_trend < 0, 1, 0)) AS cnt_no_broadcast_with_slowdown,
        -- 总播报减速数
        SUM(IF(t.is_first_broad = 1 AND t.is_sec_broad = 1 AND t.speed_trend < 0, 1, 0)) AS cnt_total_broadcast_with_slowdown,
        -- 一次播报减速数
        SUM(IF(t.is_first_broad = 1 AND (t.is_sec_broad IS NULL OR t.is_sec_broad = 0) AND t.speed_trend < 0, 1, 0)) AS cnt_first_broadcast_with_slowdown,
        -- 二次播报减速数
        SUM(IF(t.is_sec_broad = 1 AND (t.is_first_broad IS NULL OR t.is_first_broad = 0) AND t.speed_trend < 0, 1, 0)) AS cnt_second_broadcast_with_slowdown
    FROM (
        SELECT
            dt,
            city_id,
            order_id,
            driver_id,
            risk_id,
            min(speed_trend) as speed_trend,
            max(is_first_broad) as is_first_broad,
            max(is_sec_broad) as is_sec_broad
        FROM
            traffic_online.dwd_map_traffic_safety_driver_behavior_detail_di
        WHERE
            dt = "2025-11-11"
        GROUP BY
            dt, city_id, order_id, driver_id, risk_id
    ) t
    GROUP BY
        dt, city_id
) a
LIMIT 100
""").toPandas()
print(df)

            dt  city_id  city_name  cnt_no_broadcast  cnt_total_broadcast  \
0   2025-11-11      364        364               671                  107   
1   2025-11-11      260        260             13184                 1318   
2   2025-11-11       26         26            210164                 6652   
3   2025-11-11      365        365              1917                  161   
4   2025-11-11      124        124             41743                 3369   
5   2025-11-11      110        110             39768                 1983   
6   2025-11-11       35         35            394826                16356   
7   2025-11-11       21         21            676370                39662   
8   2025-11-11      143        143             79804                 4785   
9   2025-11-11      332        332                79                   11   
10  2025-11-11       46         46            145599                10585   
11  2025-11-11      306        306             46726                 2984   

25/11/14 16:13:41 ERROR YarnScheduler: Lost executor 648 on bigdata-nmg-hdp3961.nmg01.diditaxi.com: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.


#### 总体干预有效率

In [32]:
df = spark.sql("""
SELECT
    dt,
    cnt_no_broadcast,
    cnt_total_broadcast,
    cnt_first_broadcast,
    cnt_second_broadcast,
    cnt_no_broadcast_with_slowdown,
    cnt_total_broadcast_with_slowdown,
    cnt_first_broadcast_with_slowdown,
    cnt_second_broadcast_with_slowdown,
    -- 无播报减速率
    CONCAT(ROUND(IF(cnt_no_broadcast = 0, 0, cnt_no_broadcast_with_slowdown / cnt_no_broadcast * 100), 2), '%') AS no_broadcast_slow_rate,
    -- 一次播报减速率
    CONCAT(ROUND(IF(cnt_first_broadcast = 0, 0, cnt_first_broadcast_with_slowdown / cnt_first_broadcast * 100), 2), '%') AS first_broadcast_slow_rate,
    -- 二次播报减速率
    CONCAT(ROUND(IF(cnt_second_broadcast = 0, 0, cnt_second_broadcast_with_slowdown / cnt_second_broadcast * 100), 2), '%') AS second_broadcast_slow_rate,
    -- 总播报减速率
    CONCAT(ROUND(IF(cnt_total_broadcast = 0, 0, cnt_total_broadcast_with_slowdown / cnt_total_broadcast * 100), 2), '%') AS total_broadcast_slow_rate
FROM (
    SELECT
        dt,
        -- 无播报风险点数
        SUM(IF(t.is_first_broad IS NULL AND t.is_sec_broad IS NULL, 1, 0)) AS cnt_no_broadcast,
        --总播报点数
        SUM(IF(t.is_first_broad = 1 AND t.is_sec_broad = 1, 1, 0)) AS cnt_total_broadcast,
        -- 仅一次播报数
        SUM(IF(t.is_first_broad = 1 AND (t.is_sec_broad IS NULL OR t.is_sec_broad = 0), 1, 0)) AS cnt_first_broadcast,
        -- 仅二次播报数
        SUM(IF(t.is_sec_broad = 1 AND (t.is_first_broad IS NULL OR t.is_first_broad = 0), 1, 0)) AS cnt_second_broadcast,
        -- 无播报减速数
        SUM(IF(t.is_first_broad IS NULL AND t.is_sec_broad IS NULL AND t.speed_trend < 0, 1, 0)) AS cnt_no_broadcast_with_slowdown,
        -- 总播报减速数
        SUM(IF(t.is_first_broad = 1 AND t.is_sec_broad = 1 AND t.speed_trend < 0, 1, 0)) AS cnt_total_broadcast_with_slowdown,
        -- 一次播报减速数
        SUM(IF(t.is_first_broad = 1 AND (t.is_sec_broad IS NULL OR t.is_sec_broad = 0) AND t.speed_trend < 0, 1, 0)) AS cnt_first_broadcast_with_slowdown,
        -- 二次播报减速数
        SUM(IF(t.is_sec_broad = 1 AND (t.is_first_broad IS NULL OR t.is_first_broad = 0) AND t.speed_trend < 0, 1, 0)) AS cnt_second_broadcast_with_slowdown
    FROM (
        SELECT
            dt,
            city_id,
            order_id,
            driver_id,
            risk_id,
            min(speed_trend) as speed_trend,
            max(is_first_broad) as is_first_broad,
            max(is_sec_broad) as is_sec_broad
        FROM
            traffic_online.dwd_map_traffic_safety_driver_behavior_detail_di
        WHERE
            dt = "2025-11-11"
        GROUP BY
            dt, city_id, order_id, driver_id, risk_id
    ) t
    GROUP BY
        dt
) a
LIMIT 100
""").toPandas()
print("总体干预有效率")
print(df)

总体干预有效率
           dt  cnt_no_broadcast  cnt_total_broadcast  cnt_first_broadcast  \
0  2025-11-11         157685452              2046985             13123962   

   cnt_second_broadcast  cnt_no_broadcast_with_slowdown  \
0               1854384                        32510476   

   cnt_total_broadcast_with_slowdown  cnt_first_broadcast_with_slowdown  \
0                             680735                            2178456   

   cnt_second_broadcast_with_slowdown no_broadcast_slow_rate  \
0                              835753                 20.62%   

  first_broadcast_slow_rate second_broadcast_slow_rate  \
0                     16.6%                     45.07%   

  total_broadcast_slow_rate  
0                    33.26%  


In [ ]:
spark.stop()